In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

True

In [19]:
import os

os.environ['LANGCHAIN_API_KEY']='ls__c2d24a8504df472b8b0cd20d4341fa6c'
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT']="niddy-bot-agent-test"
os.environ['LANGCHAIN_TRACING_V2']='True'

In [20]:
os.getenv('LANGCHAIN_ENDPOINT')

'https://api.smith.langchain.com'

# Conversational agent using function agent

## retriever tool

In [21]:
from modules.retrievers.faiss import FaissRetriever
from modules.loaders import MyDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

COLLECTION_NAME = 'NDIS_ALL_PDFPLUMBER_TEXTS_1024_256'

loader = MyDirectoryLoader(dir_path = '../data')
splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=256)
search_kwargs = {
    "distance_metric": "cos",
    "fetch_k": 10, 
    "k": 10,
    "maximal_marginal_relevance": True
}
newCollection_kwargs = {
    "loader": loader,
    "splitter": splitter
}
retriever = FaissRetriever().build(COLLECTION_NAME, search_kwargs, newCollection_kwargs)

In [33]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    retriever, 
    "search_NDIS_Price_Guide_2023_24",
    #"Searches and returns documents regarding NDIS Support items, their maximum price for different locations and related claiming rules for claiming period starting 1 July 2023"
    "Consult the NDIS Price Guide using semantic search"
)
tools = [tool]

## agent  

### Memory

In [1]:
# https://python.langchain.com/docs/use_cases/question_answering/how_to/conversational_retrieval_agents

#----------
# MEMORY
#----------

from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-4', temperature = 0)

# This is needed for both the memory and the prompt
memory_key = "history"

memory = AgentTokenBufferMemory(memory_key=memory_key, llm=llm)




KeyboardInterrupt



### Prompt and agent

In [49]:
#-----------------
# PROMPT TEMPLATE
#-----------------

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

prompt = """
As an expert in NDIS invoicing, your role is to assist health providers in creating invoices for NDIS participants. You will answer their queries based on the NDIS Price Guide and associated documents, helping them select the appropriate item code, determine the maximum price they can charge for their services based on their location, and provide advice following the recommendations set up in the price guide for that particular service.
To do so, you any tool available. 
You may need to use search_NDIS_Price_Guide_2023_24 several times with different queries to get the information required.
When using search_NDIS_Price_Guide_2023_24, reformulating the query so that it includes the context is required.

When responding, adhere to the following guidelines:

1. If the information provided is insufficient to determine the appropriate item code, request the necessary details from the user.
2. If multiple item codes match the provided criteria, identify the distinguishing factors between these codes and ask the user to provide additional information.
4. If uncertain about NDIS Price Guide answer, take it step by step and use the NDIS Price Guide tool for the correct time period.

When providing information on how to claim for a service, structure your response as follows:

### Support Item Code
Provide the support item code and its description: $support_item_code, $description_support_item_code

### Maximum Price
Specify the maximum price that can be charged: $max_price for the location specified

### Specific Rules
Outline any additional rules and information related to the item code: $additional_rules_and_information_for_that_item_code

Remember, your role is to guide and assist. Be clear, concise, and accurate in your responses.
"""

system_message = SystemMessage(
        content=(
            # "Do your best to answer the questions. "
            # "Feel free to use any tools available to look up "
            # "relevant information, only if neccessary"
            prompt
        )
)

prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
    )

#-----------------
# AGENT
#-----------------

agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)



In [50]:
#-----------------
# AGENT EXECUTOR
#-----------------

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True,
                                   return_intermediate_steps=True)

In [45]:
result = agent_executor({"input": "What item code should I use for mowing the lawn of a participant in Pennant Hills NSW?"})



> Entering new AgentExecutor chain...

Invoking: `search_NDIS_Price_Guide_2023_24` with `lawn mowing service`


[Document(page_content='its maintenance. Includes assisting participant to do basic house and yard work.\nThis support item can be delivered to individual participants subject to the rules set out in the NDIS\nPricing Arrangements and Price Limits.\nAs well as direct service provision, this support item can be used to claim for:\n• Non-Face-to-Face Support Provision\n• Provider Travel\n• Short Notice Cancellations.\nNDIS Pricing Arrangements and Price Limits 2023-24 Version 1.0 (published 16/06/2023) Page 40 of 101', metadata={'source': '../data/PB NDIS Pricing Arrangements and Price Limits 2023-24 .pdf', 'file_path': '../data/PB NDIS Pricing Arrangements and Price Limits 2023-24 .pdf', 'page': 40, 'total_pages': 101, 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_Enabled': 'true', 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_SetDate': '2022-02-24T21:20:38Z', 'MSIP_La

In [46]:
result

{'input': 'What item code should I use for mowing the lawn of a participant in Pennant Hills NSW?',
 'history': [HumanMessage(content='What item code should I use for mowing the lawn of a participant in Pennant Hills NSW?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_NDIS_Price_Guide_2023_24', 'arguments': '{\n  "__arg1": "lawn mowing service"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=\'its maintenance. Includes assisting participant to do basic house and yard work.\\nThis support item can be delivered to individual participants subject to the rules set out in the NDIS\\nPricing Arrangements and Price Limits.\\nAs well as direct service provision, this support item can be used to claim for:\\n• Non-Face-to-Face Support Provision\\n• Provider Travel\\n• Short Notice Cancellations.\\nNDIS Pricing Arrangements and Price Limits 2023-24 Version 1.0 (published 16/06/2023) Page 40 of 101\', m

In [51]:
result2 = agent_executor({"input": "OK another query. I am working to support group-based activity (bike riding) for 5 participants. When can I charge non face 2 face support and what documentation do I need to keep?"})



> Entering new AgentExecutor chain...

Invoking: `search_NDIS_Price_Guide_2023_24` with `group-based activity bike riding non face to face support`


[Document(page_content='Example:\nFarhana attends a group program at a centre. She attends for 2 hours a week, with\n2 other participants. They all have an interest in bikes and bike riding, and as a\ngroup, have been learning how to maintain bikes with their support worker, Sunny.\nFarhana typically pays for the 2 hours of face-to-face support, 2 hours of centre\ncapital costs, and what works out to be about 10 minutes’ worth of agreed NF2F\nsupport.\nSunny notices a local bike riding club is having an open day. The group discuss\nthe club and open day over a few sessions. As the open day approaches, Farhana\ndecides she would like to attend but doesn’t feel confident approaching the club\nindependently.\nThe provider and Farhana agree for Sunny to spend up to an hour working with the\nclub to plan for Farhana to attend the open day.\n

In [52]:
result3 = agent_executor({"input": "What does this ratio mean?"})



> Entering new AgentExecutor chain...
The ratio in the context of NDIS services, such as 1:2, refers to the number of support workers to participants. 

In this case, a 1:2 ratio means one support worker is providing services to two participants. This is often used in group-based activities or supports where a single support worker is able to assist multiple participants at the same time. 

The ratio can vary depending on the type of support and the needs of the participants. For example, some activities may require a 1:1 ratio (one support worker per participant) if the participant needs more individualized attention, while other activities may have a higher ratio like 1:3 or 1:4 if the activity can be conducted in a group setting with less direct supervision. 

The ratio is important as it can affect the pricing of the service. Services with a lower ratio (more individualized attention) are generally priced higher than those with a higher ratio (less individualized attention).

> F

In [59]:
memory.chat_memory.messages

[HumanMessage(content='OK another query. I am working to support group-based activity (bike riding) for 5 participants. When can I charge non face 2 face support and what documentation do I need to keep?', additional_kwargs={}, example=False),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_NDIS_Price_Guide_2023_24', 'arguments': '{\n  "__arg1": "group-based activity bike riding non face to face support"\n}'}}, example=False),
 FunctionMessage(content='[Document(page_content=\'Example:\\nFarhana attends a group program at a centre. She attends for 2 hours a week, with\\n2 other participants. They all have an interest in bikes and bike riding, and as a\\ngroup, have been learning how to maintain bikes with their support worker, Sunny.\\nFarhana typically pays for the 2 hours of face-to-face support, 2 hours of centre\\ncapital costs, and what works out to be about 10 minutes’ worth of agreed NF2F\\nsupport.\\nSunny notices a local bike riding club is having a

In [54]:
result4 = agent_executor({"input": "So in my case the ratio is 1:5 how does that change the support item code?"})



> Entering new AgentExecutor chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7ff55c88b8d0>: Failed to resolve 'api.openai.com' ([Errno -5] No address associated with hostname)")).



Invoking: `search_NDIS_Price_Guide_2023_24` with `group-based activity bike riding support item code for 1:5 ratio`


[Document(page_content='Support Item Support Item Ref No. Description UOM Quote Required\nBicycle - Adapted for Hand Propulsion 05_121805111_0112_1_2 Hand-propelled, two-wheeled cycles for an individual with functional limitations. Each Y\nBicycle - Tricycle and/or Carts 05_121800821_0112_1_2 Bicycles, tricycles adapted for specific functional needs - primarily for children Each N\nRecreation AT - Game Interface 05_300309811_0112_1_2 Interfaces that allow a person with disability to access recreational gaming equipment when Each N\nunable to do so through conventional routes.\nBicycle - Adaptation for Pedals / Seat / 05_121821871_0112_1_2 Adaptions to existing bicycle or tricycle to address disability requirements of the participant. Each N\nHandle Bars\nEach N\nNDIS ATHM and Consumables Code Guide 2023-24 Version 1.0 Page 31 of 35', metadata={'source': '../data/PB NDI

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-FEVok5xAYTiY2dglU0PBfX6i on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-FEVok5xAYTiY2dglU0PBfX6i on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-FEVok5xAYTiY2dglU0PBfX6i on tokens p

InvalidRequestError: This model's maximum context length is 8192 tokens. However, your messages resulted in 10558 tokens (10508 in the messages, 50 in the functions). Please reduce the length of the messages or functions.

# React agent

In [4]:
#-------------------------------------------------------------------------------------------------------------------------------------
# from https://python.langchain.com/docs/modules/agents/agent_types/react.html
#-------------------------------------------------------------------------------------------------------------------------------------

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

tools = load_tools(["serpapi", "llm-math"], llm=llm)



In [5]:
from langchain.tools.render import render_text_description
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain import hub

In [12]:
prompt = hub.pull("hwchase17/react")
print('1:', prompt)
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
print('2:', prompt)

1: input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] output_parser=None partial_variables={} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}' template_format='f-string' validate_template=True
2: input_variables=['input', 'agent_scratchpad'] output_parser=None partial_variables={'tools': 'Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculat

In [8]:
llm_with_stop = llm.bind(stop=["\nObservation"])

In [9]:
"""Agent: takes in an input and produces an action and action input."""

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps'])
} | prompt | llm_with_stop | ReActSingleInputOutputParser()

In [14]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"})



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"[{'title': "Leonardo DiCaprio, 48, and new girlfriend Vittoria Ceretti, 25, hang out with actor's mom in Milan", 'link': 'https://pagesix.com/2023/09/25/leonardo-dicaprio-and-new-girlfriend-vittoria-ceretti-hang-out-with-actors-mom/', 'source': 'Page Six', 'date': '9 hours ago', 'thumbnail': 'https://serpapi.com/searches/6512a5dc49ecdb9f83f11cc3/images/42b85d3aa236916e60058b19b20af25c454517950019f8bc.jpeg'}, {'title': "Leonardo DiCaprio, 48, is 'getting serious' with Vittoria Ceretti, 25, as the Italian model has become his 18T", 'link': 'https://www.dailymail.co.uk/tvshowbiz/article-12558171/Leonardo-DiCaprio-48-getting-Vittoria-Ceretti-25-Italian-model-18TH-GIRLFRIEND.html', 'source': 'Daily Mail', 'date': '18 hours ago', 'thumbnail': 'https://serpapi.com/searches/6512a5dc49ecdb9f83f11cc

{'input': "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
 'output': 'Vittoria Ceretti is 25 years old and her age raised to the 0.43 power is 3.991298452658078.'}

# Talk to your csv 

In [17]:
from langchain import OpenAI
from langchain.agents import create_pandas_dataframe_agent
import pandas as pd
import json
import matplotlib.pyplot as plt

def create_agent(filename: str):
    """
    Create an agent that can access and use a large language model (LLM).

    Args:
        filename: The path to the CSV file that contains the data.

    Returns:
        An agent that can access and use the LLM.
    """

    # Create an OpenAI object.
    llm = OpenAI()

    # Read the CSV file into a Pandas DataFrame.
    df = pd.read_csv(filename)

    # Create a Pandas DataFrame agent.
    return create_pandas_dataframe_agent(llm, df, verbose=False)

def query_agent(agent, query):
    """
    Query an agent and return the response as a string.

    Args:
        agent: The agent to query.
        query: The query to ask the agent.

    Returns:
        The response from the agent as a string.
    """

    prompt = (
        """
            For the following query, if it requires drawing a table, reply as follows:
            {"table": {"columns": ["column1", "column2", ...], "data": [[value1, value2, ...], [value1, value2, ...], ...]}}

            If the query requires creating a bar chart, reply as follows:
            {"bar": {"columns": ["A", "B", "C", ...], "data": [25, 24, 10, ...]}}

            If the query requires creating a line chart, reply as follows:
            {"line": {"columns": ["A", "B", "C", ...], "data": [25, 24, 10, ...]}}

            There can only be two types of chart, "bar" and "line".

            If it is just asking a question that requires neither, reply as follows:
            {"answer": "answer"}
            Example:
            {"answer": "The title with the highest rating is 'Gilead'"}

            If you do not know the answer, reply as follows:
            {"answer": "I do not know."}

            Return all output as a string.

            All strings in "columns" list and data list, should be in double quotes,

            For example: {"columns": ["title", "ratings_count"], "data": [["Gilead", 361], ["Spider's Web", 5164]]}

            Lets think step by step.

            Below is the query.
            Query: 
            """
        + query
    )

    # Run the prompt through the agent.
    response = agent.run(prompt)

    # Convert the response to a string.
    return response.__str__()

def decode_response(response: str) -> dict:
    """This function converts the string response from the model to a dictionary object.

    Args:
        response (str): response from the model

    Returns:
        dict: dictionary with response data
    """
    return json.loads(response)

def write_response(response_dict: dict):
    """
    Write a response from an agent to a Streamlit app.

    Args:
        response_dict: The response from the agent.

    Returns:
        None.
    """

    # Check if the response is an answer.
    if "answer" in response_dict:
        print(response_dict["answer"])
    # Check if the response is a bar chart.
    if "bar" in response_dict:
        data = response_dict["bar"]
        df = pd.DataFrame(data)
        df.set_index("columns", inplace=True)
        df.plot(kind='bar')
        plt.show()

    # Check if the response is a line chart.
    if "line" in response_dict:
        data = response_dict["line"]
        df = pd.DataFrame(data)
        df.set_index("columns", inplace=True)
        df.plot(kind='line')
        plt.show()

    # Check if the response is a table.
    if "table" in response_dict:
        data = response_dict["table"]
        df = pd.DataFrame(data["data"], columns=data["columns"])
        df.head()

# Agent with llama index retriever

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [5]:
!pip install llama-index

  Obtaining dependency information for llama-index from https://files.pythonhosted.org/packages/c8/29/5507449a06e4af19aa1965ee8ef9368d163feaa1f1ffab38e10017ddd073/llama_index-0.8.36-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<2 from https://files.pythonhosted.org/packages/c5/05/c214b32d21c0b465506f95c4f28ccbcba15022e000b043b72b3df7728471/urllib3-1.26.16-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 957.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.4 MB/s eta 0:00:00:00:0100:01
  Obtaining dependency information for soupsieve>1.2 from https://files.pythonhosted.org/packages/4c/f3/038b302fdfbe3be7da016777069f26ceefe11a681055ea1f7817546508e3/soupsieve-2.5-py3-none-any.whl.metadata
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8

In [8]:
!pip install pypdf

  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/bf/53/8840f93c5dcd108c02cac7343e194f9dc5d15ade6200ccc661ab4e1352b5/pypdf-3.16.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 5.1 MB/s eta 0:00:00a 0:00:01


## setting up retriever as tool

In [3]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from llama_index import VectorStoreIndex, SimpleDirectoryReader

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


### Loader

In [4]:
documents = SimpleDirectoryReader("../datatest2").load_data()


DEBUG:llama_index.readers.file.base:> [SimpleDirectoryReader] Total files added: 1
> [SimpleDirectoryReader] Total files added: 1
LLAMINDEX Input file: ../datatest2/PB NDIS Pricing Arrangements and Price Limits 2023-24 .pdf
PDF reader opening file  ../datatest2/PB NDIS Pricing Arrangements and Price Limits 2023-24 .pdf
Core – Assistance with Daily Life  
NDIS Pricing Arrangements and Price Limits 202 3-24 Version 1.0  (published 16/06/2023 )  Page 40 of 101 Assistance with Self Care Activities – Night-Time Sleepover Support  
This support item provides a participant with assistance with, or supervision of, personal tasks of 
daily life where overnight support is needed, but the caregiver can sleep when not required to 
provide support. This support applies to any day of the week and on public holidays. This support 
item includes up to two hours of active supports provided to the participant for the duration of the 
period. Providers may claim for the third or additional hour at Saturd

In [5]:
prompt2 = """
As an expert in NDIS invoicing, your role is to assist health providers in creating invoices for NDIS participants. 
You will help them select the appropriate support item code, determine the maximum price they can charge for their services based on their location, and provide advice following the recommendations set up in the price guide for that particular service.
To do so, use any tool available.

When responding, adhere to the following guidelines:

1. If the information provided is insufficient to determine the support item code, request the necessary details from the user.
2. If multiple item codes match the provided criteria, identify the distinguishing factors between these codes and ask the user to provide additional information.
4. Don't refer the user to the NDIS Price Guide, instead try to use a tool to find the information.

When providing information on how to claim for a service, structure your response as follows:

### Support Item Code
Provide the support item code and its description: $support_item_code, $description_support_item_code

### Maximum Price
Specify the maximum price that can be charged: $max_price for the location specified

### Specific Rules
Outline specific claiming rules related to the support item code or support category, and remind the human about the general claiming rule when required.

Remember, your role is to guide and assist. Be clear, concise, and accurate in your responses.
"""

In [6]:

from llama_index import ServiceContext, LLMPredictor, OpenAIEmbedding, PromptHelper
from llama_index.llms import OpenAI
from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser

llm = ChatOpenAI(model='gpt-4', temperature=0)
embed_model = OpenAIEmbedding()
node_parser = SimpleNodeParser.from_defaults(
  text_splitter=TokenTextSplitter(chunk_size=2048, chunk_overlap=128))

service_context = ServiceContext.from_defaults(
  llm=llm,
  embed_model=embed_model,
  node_parser=node_parser, 
  system_prompt=prompt2
)

index = VectorStoreIndex.from_documents(documents=documents, show_progress=False,service_context=service_context)

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: National Disability Insurance Scheme  
Pricing ...
> Adding chunk: National Disability Insurance Scheme  
Pricing ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: NDIS Pricing Arrangements and Price Limits 202 ...
> Adding chunk: NDIS Pricing Arrangements and Price Limits 202 ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: NDIS Pricing Arrangements and Price Limits 2023...
> Adding chunk: NDIS Pricing Arrangements and Price Limits 2023...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: NDIS Pricing Arrangements and Price Limits 202 ...
> Adding chunk: NDIS Pricing Arrangements and Price Limits 202 ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: NDIS Pricing Arrangements and Price Limits 202 ...
> Adding chunk: NDIS Pricing Arrangements and Price Limits 202 ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: NDIS Pricing Arrangements and Price Limits 202 ...
> Adding chunk: N

In [22]:

from pprint import pprint
#pprint(documents[39].text)

### tool

In [7]:

tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about the NDIS Price Guide and claiming for services to NDIS participants. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

### Langchain memory and llm for agent 

In [13]:

#----------------------
# MEMORY and LLM
#----------------------

from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-4', temperature = 0)

# This is needed for both the memory and the prompt
memory_key = "history"
memory = AgentTokenBufferMemory(memory_key=memory_key, llm=llm)

In [14]:
#-----------------
# PROMPT TEMPLATE
#-----------------

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
# from langchain.agents import ConversationalChatAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

prompt = """
As an expert in NDIS invoicing, your role is to assist health providers in creating invoices for NDIS participants. 
You will help them select the appropriate support item code, determine the maximum price they can charge for their services based on their location, and provide advice following the recommendations set up in the price guide for that particular service.
To do so, use any tool available.

When responding, adhere to the following guidelines:

1. If the information provided is insufficient to determine the support item code, request the necessary details from the user.
2. If multiple item codes match the provided criteria, identify the distinguishing factors between these codes and ask the user to provide additional information.
4. Don't refer the user to the NDIS Price Guide, instead try to use a tool to find the information.

When providing information on how to claim for a service, structure your response as follows:

### Support Item Code
Provide the support item code and its description: $support_item_code, $description_support_item_code

### Maximum Price
Specify the maximum price that can be charged: $max_price for the location specified

### Specific Rules
Outline specific claiming rules related to the support item code or support category, and remind the human about the general claiming rule when required.

Remember, your role is to guide and assist. Be clear, concise, and accurate in your responses.
"""

system_message = SystemMessage(
        content=(
            # "Do your best to answer the questions. "
            # "Feel free to use any tools available to look up "
            # "relevant information, only if neccessary"
            prompt
        )
)

prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
    )

#-----------------
# AGENT
#-----------------

agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

In [15]:
#-----------------
# AGENT EXECUTOR
#-----------------

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True,
                                   return_intermediate_steps=True)

In [40]:
agent.plan

<member '_lc_kwargs' of 'Serializable' objects>

In [16]:
agent_executor({"input": "hi, i am bob"})



> Entering new AgentExecutor chain...
format_message: variable_name:  history
format_message: value  [] <class 'list'>
format_message: variable_name:  agent_scratchpad
format_message: value  [] <class 'list'>
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "system", "content": "\\nAs an expert in NDIS invoicing, your role is to assist health providers in creating invoices for NDIS participants. \\nYou will help them select the appropriate support item code, determine the maximum price they can charge for their services based on their location, and provide advice following the recommendations set up in the price guide for that particular service.\\nTo do so, use any tool available.\\n\\nWhen responding, adhere to the following guidelines:\\n\\n1. If the information provided is ins

{'input': 'hi, i am bob',
 'history': [HumanMessage(content='hi, i am bob', additional_kwargs={}, example=False),
  AIMessage(content='Hello Bob! How can I assist you today with NDIS invoicing?', additional_kwargs={}, example=False)],
 'output': 'Hello Bob! How can I assist you today with NDIS invoicing?',
 'intermediate_steps': []}

In [17]:
agent_executor({"input":"I have mowed the lawn for a participant on saturday morning in Pennant Hills NSDW, what is the support item code that I should use?"})



> Entering new AgentExecutor chain...
format_message: variable_name:  history
format_message: value  [HumanMessage(content='hi, i am bob', additional_kwargs={}, example=False), AIMessage(content='Hello Bob! How can I assist you today with NDIS invoicing?', additional_kwargs={}, example=False)] <class 'list'>
format_message: variable_name:  agent_scratchpad
format_message: value  [] <class 'list'>
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "system", "content": "\\nAs an expert in NDIS invoicing, your role is to assist health providers in creating invoices for NDIS participants. \\nYou will help them select the appropriate support item code, determine the maximum price they can charge for their services based on their location, and provide advice following the recommendations 

{'input': 'I have mowed the lawn for a participant on saturday morning in Pennant Hills NSDW, what is the support item code that I should use?',
 'history': [HumanMessage(content='hi, i am bob', additional_kwargs={}, example=False),
  AIMessage(content='Hello Bob! How can I assist you today with NDIS invoicing?', additional_kwargs={}, example=False),
  HumanMessage(content='I have mowed the lawn for a participant on saturday morning in Pennant Hills NSDW, what is the support item code that I should use?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'LlamaIndex', 'arguments': '{\n  "__arg1": "lawn mowing service for a participant in Pennant Hills NSDW"\n}'}}, example=False),
  FunctionMessage(content='### Support Item Code\nThe support item code for lawn mowing service falls under House or Yard Maintenance, which is 01_019_0120_1_1.\n\n### Maximum Price\nThe maximum price that can be charged for this service in a non-remote 

In [17]:
agent_executor.run(input="Can you outline what are the possible item codes? i.e. if line item is this, then support item code is that etc ")

'The possible support item codes for mowing the lawn are not provided in the given context information.'